In [2]:
import matplotlib.pyplot as plt
from t4gpd.io.CirReader import CirReader
#from t4gpd.pyvista.ToUnstructuredGrid import ToUnstructuredGrid
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
import geopandas

In [3]:
gdf = CirReader("/Users/aogecui/Desktop/实习项目/Re_ Reunion bilan GLO dans le SIG (2)/scene_masque.cir").run()

In [4]:
basemap_x=[]
basemap_y=[]
basemap_z=[]
sidewall_x=[]
sidewall_y=[]
sidewall_z=[]
for i in range(len(gdf)):
    f = gdf.geometry[i]
    fig = Polygon(f)
    x, y, z=zip(*fig.exterior.coords)
    #if sum(z)==0:
    
    if np.std(z)==0:
        basemap_x.append(x)
        basemap_y.append(y)
        basemap_z.append(z)
    else:
        sidewall_x.append(x)
        sidewall_y.append(y)
        sidewall_z.append(z)

In [5]:
elevation = []
for i in range(len(basemap_z)):
    elevation.append(np.mean(basemap_z[i]))
df = GeoDataFrame({'Elevation':elevation})
df['gid'] = df.index+1

In [6]:
#lable
lable = []
for i in range(len(df)):
    if df.Elevation[i]==0:
        lable.append("Ground")
    else:
        lable.append("Building")
df['Lable'] = lable

In [7]:
#geometry
geometry =[]
for i in range(len(df)):
    p = Polygon(list(zip(basemap_x[i],basemap_y[i])))
    geometry.append(p)
df['geometry'] = geometry

In [8]:
#floor
floor = []
for i in range(len(df)):
    n=0
    if df.Lable[i] == 'Ground':
        floor.append(0)
    else:
        for j in range(len(sidewall_x)):
            q = Polygon(list(zip(sidewall_x[j],sidewall_y[j])))
            if df.geometry[i].contains(q):
                n = n+1
        nmb = len(basemap_x[i])-1
        floor.append(n/nmb)
df['floor'] = floor
df

,Elevation,gid,Lable,geometry,floor
0,0.0,1,Ground,"POLYGON ((-30.000 -30.000, -30.000 120.000, 12...",0.0
1,18.0,2,Building,"POLYGON ((70.000 50.000, 70.000 70.000, 50.000...",6.0
2,18.0,3,Building,"POLYGON ((25.000 0.000, 25.000 20.000, 5.000 2...",6.0
3,30.0,4,Building,"POLYGON ((20.000 65.000, 40.000 65.000, 40.000...",10.0
4,30.0,5,Building,"POLYGON ((95.000 70.000, 95.000 90.000, 75.000...",10.0
5,30.0,6,Building,"POLYGON ((65.000 5.000, 65.000 25.000, 85.000 ...",10.0
6,36.0,7,Building,"POLYGON ((60.000 80.000, 60.000 100.000, 40.00...",12.0
7,42.0,8,Building,"POLYGON ((100.000 60.000, 100.000 40.000, 80.0...",14.0
8,42.0,9,Building,"POLYGON ((20.000 75.000, 20.000 95.000, 0.000 ...",14.0
9,24.0,10,Building,"POLYGON ((35.000 40.000, 35.000 20.000, 55.000...",8.0
